<a href="https://colab.research.google.com/github/abdallaho98/AutoSpeech/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 1.3MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 983kB 52.2MB/s 
     |████████████████████████████████| 245kB 53.3MB/s 
     |████████████████████████████████| 778kB 44.7MB/s 
     |████████████████████████████████| 471kB 51.1MB/s 
     |████████████████████████████████| 450kB 51.3MB/s 
     |████████████████████████████████| 3.9MB 48.1MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 1.1MB 47.5MB/s 
     |████████████████████████████████| 3.0MB 51.4MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.8-cp36-none-any.whl size=25268462 sha256=1e274f269a57e7227186f0c9e2b7f1c757c890a655cb4c0daf103ae69181d1b6
  Stored in directory: /root/.cache/pip/wheels/53/0d/49/76e18efeba19337a3dd626260325525e61c2e83974d6bbeae4
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [7]:
import ktrain
from ktrain import text

In [8]:
import pandas as pd
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('type', 'content'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       df.loc[cnt] = [int(x[1]) , x[0]]
       line = fp.readline()
       cnt += 1
df['type'] = df['type'].apply(lambda x: 'right' if x == 1 else 'permit' if x == 2 else 'obligation' if x == 3 else 'prohibition' if x == 4 else 'other')
df.columns = ['label', 'text']
df = pd.concat([df, df.label.astype('str').str.get_dummies()], axis=1, sort=False)
df = df[['text', 'right', 'permit','obligation' ,'prohibition' , 'other']]
df.head()

,text,right,permit,obligation,prohibition,other
0,ليس للشركاء في مال شائع أن يطلبوا قسمته إذا تب...,0,0,0,1,0
1,إذا وقع تحريف في نص السفتجة فالأشخاص الموقعون ...,0,0,0,0,1
2,يجوز الحجز على الأجر أو المرتب بقيمة النفقة ال...,0,1,0,0,0
3,يعين القاضي طبقا لأحكام قانون الأسرة، مقدما من...,0,0,1,0,0
4,يجوز للعموم المعارضة في أي تصحيح قضائي أو إدار...,0,1,0,0,0


In [9]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, 
                                                                   'text', # name of column containing review text
                                                                   label_columns=['right', 'permit','obligation' ,'prohibition' , 'other'],
                                                                   maxlen=500, 
                                                                   preprocess_mode='bert')

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: ar


Is Multi-Label? False
preprocessing test...
language: ar


In [10]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner.fit_onecycle(2e-3, 3)



begin training using onecycle policy with max lr of 0.002...
Train on 931 samples, validate on 104 samples
Epoch 1/3
 24/931 [..............................] - ETA: 1:34:31 - loss: 2.0075 - accuracy: 0.2083